In [22]:
import numpy as np
import pandas as pd
import pickle
import math
import pandas as pd
from pandas import HDFStore 
import argparse

###################################################################################
#location
node_ids_filename = 'data/node_locate.txt'
with open(node_ids_filename) as f:
    _node_ids = f.read().strip()
    _node_ids = _node_ids.replace('\n', ' ')
    _node_ids = _node_ids.split(' ')

node_id = []
node_ids = []
node_name = []
node_loc = []
for i in range(len(_node_ids)):
    if(_node_ids[i] == ''):
        continue
    node_id.append(_node_ids[i])

for i in range(len(node_id)):
    if(i <= 4):
        continue
    if(node_id[i] == 'C'):
        continue
    node_ids.append(node_id[i])
for i in range(len(node_ids)):
    if(i % 4 == 0):
        line = []
        node_name.append(node_ids[i])
        line.append(node_ids[i+1])
        line.append(node_ids[i+2])
        line.append(node_ids[i+3])
        node_loc.append(line)
node_loc=np.array(node_loc).astype('float32')
print("node_loc",node_loc.shape)

###################################################################################
#save to npy file
np.save('data/node_signal', _node_signal)
np.save('data/node_loc', node_loc)

_node_signal (135, 106, 7)
node_loc (106, 3)
-118.45 139.457
-31.3102 21.6857
-563.119 91.0359
-0.0160272 0.000982589
-0.0147581 0.0163793
-0.00474041 0.0034676
0.0 0.0


In [ ]:
########################################
normalized_k=0.1
output_pkl_filename='data/sensor_graph/adj_mat.pkl'
########################################


####################################################################################
#getting distance matrix
node_loc=np.load('data/node_loc.npy')
n_nodes=node_loc.shape[0]
node_id = [str(i) for i in range(1,1+n_nodes)]
node_id_to_ind = {}
for i, node in enumerate(node_id, 1):
    node_id_to_ind[node] = i
num_node = len(node_id)
dist_mx = np.zeros((num_node, num_node), dtype=np.float32)
dist_mx[:] = np.inf
link_mx = np.zeros((num_node, num_node), dtype=np.float32)
for i in range(num_node):
    for j in range(num_node):
        x = float(node_loc[i][0]) - float(node_loc[j][0])
        y = float(node_loc[i][1]) - float(node_loc[j][1])
        z = float(node_loc[i][2]) - float(node_loc[j][2])
        x = math.pow(x,2)
        y = math.pow(y,2)
        z = math.pow(z,2)
        dis = math.sqrt(x+y+z)
        dist_mx[i, j] = dis

####################################################################################
#getting adj matrix
sensor_ids=node_id

# Builds sensor id to index map.
sensor_id_to_ind = {}
for i, sensor_id in enumerate(sensor_ids):
    sensor_id_to_ind[sensor_id] = i

# Calculates the standard deviation as theta.
distances = dist_mx[~np.isinf(dist_mx)].flatten()
std = distances.std()
adj_mx = np.exp(-np.square(dist_mx / std))
# Make the adjacent matrix symmetric by taking the max.
# adj_mx = np.maximum.reduce([adj_mx, adj_mx.T])

# Sets entries that lower than a threshold, i.e., k, to zero for sparsity.
adj_mx[adj_mx < normalized_k] = 0
#return sensor_ids, sensor_id_to_ind, adj_mx